In [1]:
from datetime import datetime, timedelta
import pandas as pd
from tqdm import tqdm
from processor.processor import Processor as p
from database.market import Market
from strats.btc_spec import BTCSpec
import numpy as np

In [2]:
market = Market()

In [3]:
training_year = 4
strat_class = BTCSpec(training_year)

In [4]:
factors = [str(x) for x in range(14)]
factors.append("d1")
factors.append("d2")
factors.append("d3")

In [5]:
included_columns=[x for x in factors]
included_columns.extend(['year',"date","ticker","y"])

In [6]:
if not strat_class.check_factors():
    print("init factors")
    strat_class.assign_factors(factors)
if not strat_class.check_included_columns():
    print("init included_columns")
    strat_class.assign_included_columns(included_columns)

init factors
init included_columns


In [7]:
strat_class.pull_included_columns()
strat_class.pull_factors()

In [8]:
market.connect()
prices = market.retrieve("pdr_crypto")
market.disconnect()

In [9]:
prices = p.column_date_processing(prices)

E:\projects\longshot_venv\longshot\processor\processor.py:19: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data["week"] = data["date"].dt.week


In [10]:
prices

,date,high,low,open,close,volume,adjclose,ticker,year,quarter,week
0,2014-09-17,468.174011,452.421997,465.864014,457.334015,2.105680e+07,457.334015,BTC,2014,3,38
1,2014-09-18,456.859985,413.104004,456.859985,424.440002,3.448320e+07,424.440002,BTC,2014,3,38
2,2014-09-19,427.834991,384.532013,424.102997,394.795990,3.791970e+07,394.795990,BTC,2014,3,38
3,2014-09-20,423.295990,389.882996,394.673004,408.903992,3.686360e+07,408.903992,BTC,2014,3,38
4,2014-09-21,412.425995,393.181000,408.084991,398.821014,2.658010e+07,398.821014,BTC,2014,3,38
...,...,...,...,...,...,...,...,...,...,...,...
2898,2022-08-19,23180.912109,22760.619141,23180.912109,22821.220703,2.595380e+10,22821.220703,BTC,2022,3,33
2899,2022-08-17,24407.058594,23243.353516,23881.316406,23335.998047,3.093162e+10,23335.998047,BTC,2022,3,33
2900,2022-08-18,23563.832031,23177.601562,23341.039062,23212.738281,2.374761e+10,23212.738281,BTC,2022,3,33
2901,2022-08-19,23213.312500,20868.847656,23213.312500,20877.552734,4.050961e+10,20877.552734,BTC,2022,3,33


In [11]:
market.connect()
ticker = "BTC"
data = []
try:
    ticker_data = prices.copy()
    ticker_data = strat_class.transform(ticker_data,ticker)
    strat_class.db.connect()
    strat_class.db.store("data",ticker_data[strat_class.included_columns])
    strat_class.db.disconnect()
except:
    print("whut")
market.disconnect()